In [1]:
proj_path = '/home/ajhnam/projects/hidden_singles_public/'

In [2]:
import sys
sys.path.append(proj_path + 'python/')

import random
import numpy as np
import itertools
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.bernoulli import Bernoulli
from torch.distributions.categorical import Categorical
from torch.distributions.uniform import Uniform
from torch.utils.data import DataLoader as DataLoader

from hiddensingles.misc import torch_utils as tu
from hiddensingles.misc import utils, TensorDict, TensorDictDataset, nnModule, MLP
from hiddensingles.experiment.sudoku_hs_service import create_tutorial, create_phase1, create_phase2

In [67]:
device = 0

In [68]:
def load_responses(puzzle_data_path, subject_data_path, solvers=True, device='cpu'):
    """
    Returns a tensor of shape [num_solvers, num_trials]
    containing the response types of each subject for each trial
    """
    puzzle_df = pd.read_csv(proj_path + puzzle_data_path, sep='\t')
    subject_df = pd.read_csv(proj_path + subject_data_path, sep='\t')
    subject_df = subject_df[~subject_df.solver.isna()]
    subject_df.solver = subject_df.solver.astype(bool)
    subject_df = subject_df[subject_df.solver if solvers else ~subject_df.solver]
    subject_ids = set(subject_df.subject_id)

    response_map = {'inhouse': 0,
                    'absent': 1,
                    'distractor': 2,
                    'target': 3}
    puzzle_df = puzzle_df[(puzzle_df.phase == 1) & (puzzle_df.subject_id.isin(subject_ids))]
    num_trials = len(puzzle_df.trial.unique())
    puzzle_df.response_type = [response_map[response] for response in puzzle_df.response_type]
    puzzle_df = puzzle_df.sort_values(['subject_id', 'trial'])
    responses = torch.tensor(puzzle_df.response_type.to_numpy(), device=device).view(-1, num_trials)
    subject_ids = puzzle_df.subject_id.unique()
    return responses, subject_ids

def split_train_test(subject_responses):
    """
    Splits the subject responses data into an 80/20 split, where every 5 trials,
        one is randomly selected to be part of the test set.
    First trial is always part of the training set.
    
    Returns a TensorDict containing
        test_trials: tensor of shape [num_subjects, 5] containing the indices of the 5 test trials
        train_mask: tensor of shape [num_subjects, 25] containing booleans for whether or not
            the trial is part of the training set
        train_responses: tensor of shape [num_subjects, 25] where it's the same as subject_responses,
            except that the test trials now have -1
        train_oh_responses: a one-hot version of train_responses, except test trials are 0 vectors
    """
    device = subject_responses.device
    # Mask the subject responses
    test_trials = 5 * tu.batch_arange(num_subjects, 5, device=device) 
    test_trials[:,0] += torch.randint(1, 5, (num_subjects, ), device=device)
    test_trials[:,1:] += torch.randint(0, 5, (num_subjects, 4), device=device)
    train_mask = torch.ones_like(subject_responses)
    for i in range(test_trials.shape[-1]):
        tu.write_subtensors(train_mask, test_trials[:,i],
                                     torch.zeros(num_subjects, dtype=int, device=device))

    train_responses = (subject_responses * train_mask) - (1 - train_mask)
    oh_responses = tu.one_hot_encode(subject_responses)
    train_oh_responses = tu.one_hot_encode(subject_responses) * train_mask.unsqueeze(-1)
    return TensorDict(test_trials=test_trials,
                      train_mask=train_mask,
                      train_responses=train_responses,
                      train_oh_responses=train_oh_responses)

In [75]:
def get_p_strat_responses(p_lapse_ih, p_lapse_a, p_lapse_d):
    """
    Returns P(response | strategy), i.e. probabilities of each response type given strategy
    """
    device = p_lapse_ih.device
    p = torch.tensor([[3/9, 4/9, 1/9, 1/9],
                      [1,   4/6, 1/6, 1/6],
                      [1,   1,   1/2, 1/2],
                      [1,   1,   1,   1  ]], device=device)
    
    # for gradient to flow, need to assign values like this
    multipliers = [[1,          1,              1,                          1],
                   [p_lapse_ih, 1 - p_lapse_ih, 1 - p_lapse_ih,             1 - p_lapse_ih],
                   [p_lapse_ih, p_lapse_a,      1 - p_lapse_ih - p_lapse_a, 1 - p_lapse_ih - p_lapse_a],
                   [p_lapse_ih, p_lapse_a,      p_lapse_d,                  1 - p_lapse_ih - p_lapse_a - p_lapse_d]]
    multipliers = tu.make_tensor(multipliers, dtype=torch.float32, device=device)
    p = p * multipliers
    return p
    

def zero_p_transitions(p_transitions):
    assert p_transitions.shape == (4, 4)
    M = torch.zeros_like(p_transitions)
    for i in range(0, 4):
        M[i, i:] = p_transitions[i, i:].softmax(-1)
    return M

def get_p_strategies(subject_responses,
                     p_strat_trial0,
                     p_transitions):
    """
    Returns P(strategy | trial)
    First, gets P(strategy | trial=1) by solving for
        P(response | strategy) * P(strategy) = P(data)
        at trial = 1.
    Then uses transition matrix to get subsequent state distributions under Markov assumption.
    """
    num_subjects, num_trials = subject_responses.shape
    p_s = p_strat_trial0
    p_strategies = [p_s]
    
    # use transition matrix to derive P(strategy | trial)
    for i in range(num_trials-1):
        p_s = p_transitions.T.matmul(p_s)
        p_strategies.append(p_s)
    p_strategies = torch.stack(p_strategies, dim=0)
    return p_strategies

def get_p_responses(p_strat_responses, p_strategies):
    """
    Returns P(response | trial) by computing
        sum{strategy} P(response | strategy, trial) * P(strategy | trial)
    """
    return p_strategies.matmul(p_strat_responses)

# Macromodels

In [93]:
class MacroModel(nn.Module):
    
    def __init__(self,
                 subject_responses,
                 p_lapse_ih=0.1,
                 p_lapse_a=0.1,
                 p_lapse_d=0.1):
        # cap p_lapse at 20%, though it wouldn't actually reach it, just to keep things somewhat reasonable
        assert 0 < p_lapse_ih <= .2
        assert 0 < p_lapse_a <= .2
        assert 0 < p_lapse_d <= .2
        
        super().__init__()
        self.subject_responses = subject_responses
        
        self.p_lapse_ih = nn.Parameter(tu.logit(torch.tensor(5 * p_lapse_ih))) # max it at 20%
        self.p_lapse_a = nn.Parameter(tu.logit(torch.tensor(5 * p_lapse_a))) # max it at 20%
        self.p_lapse_d = nn.Parameter(tu.logit(torch.tensor(5 * p_lapse_d))) # max it at 20%
        self.p_strat_trial0 = nn.Parameter(torch.rand(4))
        self.p_transitions = nn.Parameter(torch.rand(4, 4))
        
    def get_params(self):
        return TensorDict(p_lapse_ih = self.p_lapse_ih.sigmoid() / 5,
                          p_lapse_a = self.p_lapse_a.sigmoid() / 5,
                          p_lapse_d = self.p_lapse_d.sigmoid() / 5,
                          p_strat_trial0 = self.p_strat_trial0.softmax(-1),
                          p_transitions = zero_p_transitions(self.p_transitions))
        
    def forward(self, train_mask=None):
        """
        In the forward calculation, only the first trial is used (to establish prior)
        
        train_mask: boolean tensor with shape [num_subjects, num_trials] indicating which trials should be
            used for training
            If None, all trials are used for training
        """
        num_subjects, num_trials = self.subject_responses.shape
        params = self.get_params()

        p_strat_responses = get_p_strat_responses(params.p_lapse_ih,
                                                  params.p_lapse_a,
                                                  params.p_lapse_d)
        p_strategies = get_p_strategies(self.subject_responses, params.p_strat_trial0, params.p_transitions)
        p_responses = get_p_responses(p_strat_responses, p_strategies)
        
        p_responses_exp = tu.expand_along_dim(p_responses, 0, num_subjects)
        nll = tu.nll_loss(p_responses_exp.log(), subject_responses, reduction='none')
        
        if train_mask is not None:
            nll = (nll * train_mask).sum() / train_mask.sum()
        else:
            nll = nll.mean()
        
        return TensorDict(p_strat_responses=p_strat_responses,
                          p_strategies=p_strategies,
                          p_responses=p_responses,
                          nll=nll)

In [94]:
def train_macro_model(subject_responses,
                      num_epochs=1000,
                      lr=.01,
                      train_all=False,
                      show_pbar=False,
                      print_epochs=0):
    
    model = MacroModel(subject_responses)
    model = model.to(subject_responses.device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    split_responses = split_train_test(subject_responses)
    
    iterable = range(num_epochs + 1)
    
    if show_pbar > 0:
        iterable = tqdm(iterable)
        
    for epoch in iterable:
        optimizer.zero_grad()
        results = model(None if train_all else split_responses.train_mask)
        loss = results.nll
        loss.backward()
        optimizer.step()

        if print_epochs > 0 and (epoch%print_epochs == 0 or epoch == num_epochs):
            print("Epoch: {}, NLL: {}".format(epoch, loss))
            print(results.p_strategies[0])
            model.get_params().print(round_digits=4)
            
    return model, split_responses, results

# Micromodels

In [95]:
def get_p_paths(subject_responses,
                p_strat_priors,
                p_strat_responses,
                p_transitions,
                train_mask=None,
                show_pbar=False):
    """
    Returns a TensorDict with
        paths: tensor of shape [num_paths, num_trials] indicating the strategy at each step
        p_paths: tensor of shape [num_subjects, num_paths] with probability of each path
        
    train_mask: boolean tensor of shape [num_subjects, num_trials]
        Indicates trials to include the likelihood. When false, only the prior is used.
        If train_mask is None, likelihood is used in all trials.
    """
    # all operations are done sequentially, so moving to cpu is faster
    device = subject_responses.device
    subject_responses = subject_responses.cpu()
    p_transitions = p_transitions.cpu()
    p_strat_responses = p_strat_responses.cpu()
    
    if train_mask is not None:
        train_mask = train_mask.cpu()
    
    # tensor of P(actual response | strategy) for each strategy
    # shape [num_subjects, num_trials, num_strategies]  i.e. [88, 25, 4]
    likelihoods = tu.prepend_shape(p_strat_responses, subject_responses.shape)
    likelihoods = tu.swap_dims(likelihoods, -1, -2)
    likelihoods = tu.select_subtensors(likelihoods, subject_responses)

    all_p_paths = []

    # for each subject, build P(path) using Bayes Theorem
    iterable = range(num_subjects)
    if show_pbar:
        iterable = tqdm(iterable)
    for sid in iterable:
        p_s = likelihoods[sid, 0] * p_strat_priors
        p_s = p_s / p_s.sum(-1).unsqueeze(-1)
        p_paths = p_s
        paths = [(0,), (1,), (2,), (3,)]

        for trial in range(1, num_trials):
            new_paths = []
            new_p_paths = []

            for i, strat in itertools.product(range(len(paths)), range(4)):
                path = paths[i]
                
                # reject paths that switch to a worse strategy
                # they need to be left out of the loop so that they aren't included in the normalizing sum (denominator)
                if path[-1] > strat:
                    continue

                # P(path_{t-1}) x P(s | s_{t-1}) x P(r | s)
                # Only apply likelihood if the trial is part of the training set
                p_path = p_paths[i] * p_transitions[path[-1], strat]
                if train_mask is None or train_mask[sid, trial]:
                    p_path = p_path * likelihoods[sid, trial, strat]
                new_p_paths.append(p_path)

                new_paths.append(path + (strat, ))

            paths = new_paths
            p_paths = torch.stack(new_p_paths, dim=0)
            p_paths = p_paths / p_paths.sum()

        all_p_paths.append(p_paths)
    
    all_p_paths = torch.stack(all_p_paths, dim=0)
    paths = torch.tensor(paths)
    
    oh_paths = tu.one_hot_encode(paths)
    p_strategies = tu.extend_mul(all_p_paths, oh_paths).sum(1)
    p_responses = tu.extend_mul(p_strategies, p_strat_responses).sum(2)
    return TensorDict(paths=paths,
                      probs=all_p_paths,
                      p_strategies=p_strategies,
                      p_responses=p_responses).to(device)

In [96]:
def sample_responses(micro_models, num_samples):
    """
    For each participant, takes their fitted HMM and samples a strategy path.
    Then using those strategy paths, samples responses for each trial.
    """
    paths = tu.prepend_shape(micro_models.paths, (num_samples, num_subjects))
    path_distributions = Categorical(probs = micro_models.probs)
    sample_paths = path_distributions.sample((num_samples,))
    sample_strategies = tu.select_subtensors(paths, sample_paths)
    p_strat_responses_exp = tu.prepend_shape(p_strat_responses, sample_strategies.shape)
    p_responses = tu.select_subtensors(p_strat_responses_exp, sample_strategies)
    response_distributions = Categorical(probs = p_responses)
    sample_responses = response_distributions.sample()
    sample_responses = tu.swap_dims(sample_responses, 0, 1)
    return sample_responses

def compare_actual_nll(micro_models, subject_responses, num_samples, train_mask=None):
    """
    do monte carlo to get NLL distribution of model samples for each subject
    Also returns percentiles of actual data's NLL. Returns a range for valid percentiles:
        percentile_min: % of samples with NLL < actual data's
        percentile_max: 1 - % of samples with NLL > actual data's
        percentile_sample: a random number between percentile_min and percentile_max
    """ 
    samples = sample_responses(micro_models, num_samples)

    p_responses = tu.expand_along_dim(micro_models.p_responses, 1, num_samples)
    sample_nll = tu.nll_loss(p_responses.log(), samples, reduction='none')
    actual_nll = tu.nll_loss(micro_models.p_responses.log(), subject_responses, reduction='none')
    
    td = TensorDict()
    
    if train_mask is not None:
        test_mask = 1 - train_mask
        sample_test_nll = (sample_nll * test_mask.unsqueeze(1)).sum(-1)
        actual_test_nll = (actual_nll * test_mask).sum(-1)
        percentile_min = (sample_test_nll < actual_test_nll.unsqueeze(-1)).float().mean(-1)
        percentile_max = 1 - (sample_test_nll > actual_test_nll.unsqueeze(-1)).float().mean(-1)
        percentile_sample = Uniform(percentile_min, percentile_max).sample()
        
        td.test = TensorDict(sample_nll=sample_test_nll,
                             actual_nll=actual_test_nll,
                             percentile_min=percentile_min,
                             percentile_max=percentile_max,
                             percentile_sample=percentile_sample)
    
    sample_nll = sample_nll.sum(-1)
    actual_nll = actual_nll.sum(-1)
    percentile_min = (sample_nll < actual_nll.unsqueeze(-1)).float().mean(-1)
    percentile_max = 1 - (sample_nll > actual_nll.unsqueeze(-1)).float().mean(-1)
    percentile_sample = Uniform(percentile_min, percentile_max).sample()
    
    td.all = TensorDict(sample_nll=sample_nll,
                        actual_nll=actual_nll,
                        percentile_min=percentile_min,
                        percentile_max=percentile_max,
                        percentile_sample=percentile_sample)
    return td

# Cross-validation

In [97]:
def get_cv_metrics(p_responses, subject_responses, train_mask, num_samples):
    """
    Returns NLL of actual samples and percentiles of actual sample NLL
    
    p_responses: tensor of shape [num_subjects, num_trials, 4]
    train_mask: boolean tensor of shape [num_subjects, num_trials]
    num_samples: int
    """
    num_subjects, num_trials = subject_responses.shape
    assert p_responses.shape == (num_subjects, num_trials, 4)
    assert train_mask.shape == (num_subjects, num_trials)
    
    # get actual data NLL
    actual_nll = tu.nll_loss(p_responses.log(), subject_responses, reduction='none')
    actual_nll = (actual_nll * (1 - train_mask)).sum(-1)
    
    # get sample data NLL
    samples = Categorical(p_responses).sample((num_samples, ))
    p_responses = tu.prepend_shape(p_responses, num_samples)
    sample_nll = tu.nll_loss(p_responses.log(), samples, reduction='none')
    train_mask = tu.prepend_shape(train_mask, num_samples)
    sample_nll = (sample_nll * (1 - train_mask)).sum(-1)
    
    # calculate percentile range and sample a value
    percentile_min = (sample_nll < actual_nll.unsqueeze(0)).float().mean(0)
    percentile_max = 1 - (sample_nll > actual_nll.unsqueeze(0)).float().mean(0)
    percentile = Uniform(percentile_min, percentile_max).sample()
    
    return TensorDict(actual_nll=actual_nll, sample_nll=sample_nll.T, percentile=percentile)

def make_df(metrics, model_name):
    actual_df = metrics[['actual_nll', 'percentile']].to_dataframe({0: 'sim', 1: 'subject_id'})
    sample_df = metrics[['sample_nll']].to_dataframe({0: 'sim', 1: 'subject_id', 2: 'sample_id'})
    actual_df = actual_df.rename({'actual_nll': 'nll'}, axis=1)
    sample_df = sample_df.rename({'sample_nll': 'nll'}, axis=1)
    actual_df['model'] = model_name
    sample_df['model'] = model_name
    return actual_df, sample_df

# Run code

In [125]:
# Parameters
solvers = True
num_runs = 50 # For cross-validation, how many separate models to train
num_samples = 100 # For cross-validation, how many samples from each model to draw

# Path of data files
puzzle_path = "data/processed/puzzle_data.tsv"
subject_path = "data/processed/subject_data.tsv"

In [126]:
# Load data

subject_responses, subject_ids = load_responses(puzzle_path, subject_path, solvers=solvers, device=device)
num_subjects, num_trials = subject_responses.shape

# Create save directory
group = 'solvers' if solvers else 'nonsolvers'
dirpath = proj_path + 'data/hmm/{}/'.format(group)
utils.mkdir(dirpath)

### Models without held-out data

In [127]:
# Train models

macro_model, splits, results = train_macro_model(subject_responses,
                                                 train_all=True,
                                                 num_epochs=2000,
                                                 show_pbar=True)
params = macro_model.get_params().detach()
p_strat_responses = results.p_strat_responses.detach()
micro_model = get_p_paths(subject_responses,
                          params.p_strat_trial0.cpu(),
                          p_strat_responses,
                          params.p_transitions,
                          train_mask=None,
                          show_pbar=True)

In [128]:
# P(response) and P(strategy) for each participant

p_responses = utils.to_dataframe(micro_model.p_responses, ['subject_id', 'trial', 'response_type'], 'probability')
p_responses.subject_id = [subject_ids[s] for s in p_responses.subject_id]
p_responses.to_csv(dirpath + "subject_p_responses.tsv".format(group), sep='\t', index=False)

p_strategies = utils.to_dataframe(micro_model.p_strategies, ['subject_id', 'trial', 'strategy'], 'probability')
p_strategies.subject_id = [subject_ids[s] for s in p_strategies.subject_id]
p_strategies.to_csv(dirpath + "subject_p_strategies.tsv".format(group), sep='\t', index=False)

In [129]:
# Top posterior strategy paths for each participant

p_paths, path_idx = micro_model.probs.sort(-1, descending=True)
p_paths = p_paths[:,:5]
path_idx = path_idx[:,:5]

p_paths = utils.to_dataframe(p_paths, ['subject_id', 'rank'], 'probability')
p_paths.subject_id = [subject_ids[s] for s in p_paths.subject_id]
p_paths.to_csv(dirpath + "top_paths_probs.tsv", sep='\t', index=False)

top_paths = []
for i in range(num_subjects):
    idx = path_idx[i]
    top_paths.append(micro_model.paths[idx])
top_paths = torch.stack(top_paths, dim=0)
top_paths = utils.to_dataframe(top_paths, ['subject_id', 'rank', 'trial'], 'strategy')
top_paths.subject_id = [subject_ids[s] for s in top_paths.subject_id]
top_paths.to_csv(dirpath + "top_paths.tsv", sep='\t', index=False)

### Models with held-out data for cross-validation

In [130]:
# Train models

split_responses = []
macro_models = []
micro_models = []

for i in tqdm(range(num_runs)):
    macro_model, splits, results = train_macro_model(subject_responses, num_epochs=2000)
    params = macro_model.get_params().detach()
    p_strat_responses = results.p_strat_responses.detach()
    micro_model = get_p_paths(subject_responses,
                              params.p_strat_trial0.cpu(),
                              p_strat_responses,
                              params.p_transitions,
                              train_mask=splits.train_mask)
    
    split_responses.append(splits)
    macro_models.append(macro_model)
    micro_models.append(micro_model)

In [131]:
# NLL for micro models
# Does hierarchical sampling so has a slight different procedure than other models

micro_metrics = []
for i in range(num_runs):
    comparisons = compare_actual_nll(micro_models[i],
                                     subject_responses,
                                     num_samples=num_samples,
                                     train_mask=split_responses[i].train_mask)
    td = TensorDict(actual_nll=comparisons.test.actual_nll,
                    sample_nll=comparisons.test.sample_nll,
                    percentile=comparisons.test.percentile_sample)
    micro_metrics.append(td)
micro_metrics = TensorDict.stack(micro_metrics, dim=0)

In [132]:
# NLL for priors model

macro_metrics = []
for i in range(num_runs):
    outputs = macro_models[i]()
    p_responses = tu.prepend_shape(outputs.p_responses, num_subjects)
    metrics = get_cv_metrics(p_responses, subject_responses, split_responses[i].train_mask, num_samples)
    macro_metrics.append(metrics)
macro_metrics = TensorDict.stack(macro_metrics, 0)

In [133]:
# NLL for means model
means_metrics = []
eps = 1e-3 # some train/test splits can remove all instances of a response type in a trial, making p = 0
for i in range(num_runs):
    train_responses = eps + split_responses[i].train_oh_responses.sum(0).float()
    avg_responses = train_responses / train_responses.sum(-1).unsqueeze(-1)
    p_responses = tu.prepend_shape(avg_responses, num_subjects)
    metrics = get_cv_metrics(p_responses, subject_responses, split_responses[i].train_mask, num_samples)
    means_metrics.append(metrics)
means_metrics = TensorDict.stack(means_metrics, 0)

In [134]:
# Save results

dfs = [make_df(*args) for args in [(micro_metrics, 'hmm'),
                                   (macro_metrics, 'priors'),
                                   (means_metrics, 'means')]]
actual_df = pd.concat([df[0] for df in dfs])
sample_df = pd.concat([df[1] for df in dfs])
actual_df.subject_id = [subject_ids[i] for i in actual_df.subject_id]
sample_df.subject_id = [subject_ids[i] for i in sample_df.subject_id]


actual_df.to_csv(dirpath + 'nll_actual.tsv', sep='\t', index=False)
sample_df.to_csv(dirpath + 'nll_sample.tsv', sep='\t', index=False)